In [ ]:
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from SingleDay_DrawProfile_Generator import Create_Hot_Profiles, Calculate_TMains
from Event_To_Timestep_Converter import Convert_Profile_SingleDay
import holidays

#%%-----------------------DEFINE INPUTS------------------------------------

# Set details for draw profiles
ClimateZone = 3  # Specify the CEC climate zone used in these calculations
Include_Faucet = 'Yes' # State whether ('Yes') or not ('No') to include faucets in the draw profile
Include_Shower = 'Yes' # State whether ('Yes') or not ('No') to include showers in the draw profile
Include_Clothes = 'Yes' # State whether ('Yes') or not ('No') to include clothes washers in the draw profile
Include_Dish = 'Yes' # State whether ('Yes') or not ('No') to include dish washers in the draw profile
Include_Bath = 'Yes' # State whether ('Yes') or not ('No') to include baths in the draw profile
Version = 2019 # State which version of the Title 24 draw profiels to use (2016 or 2019 at this time)
Temperature_Bath = 105 # State the temperature of hot water occupants use when taking baths. deg F
Temperature_Shower = 105 # State the temperature of hot water occupants use when taking showers. deg F
SquareFootage_Dwelling = 1897 # State the conditioned floor area of the dwelling. ft^2
Water = 'Hot' # 'Hot' water from the water heater or 'Mixed' water from the fixture
Distribution_System_Type = 'Trunk and Branch' # The type of distribution system taking water to the fixtures
Timestep = 3600 # Timestep to use when converting profiles to timestep-based. Seconds
# cold water coming into heater
# daily be a year
# hourley for a whole year
# 8760 hours
# weekday and weekends 2019 values
# days for weekday and weekends also taken into account
# a sperate column for the 10 different variants
# a seperate file for different number of occupants
# the D,E and H together
# one file for ALL 4 person housholds
# draw profile is what important (L)
# 6D4, 6 how many people, 4 is 4 variants different houses (4th of the set)
# 6X1 INDIVIDUL 

Daily_Profiles = pd.read_csv(os.path.join(os.getcwd(), 'SourceData', '2019', 'DailyProfiles.csv'), index_col = 0)
Profile_List = np.unique(Daily_Profiles.index)
print(Profile_List)

T_Mains = Calculate_TMains(ClimateZone) # Calcualte the inlet water temperature. Modify to enable nationwide calculations

create_one_profile = lambda Profile, Day_Of_Year: Create_Hot_Profiles('Single', Profile, ClimateZone, Include_Faucet, Include_Shower, Include_Clothes, Include_Dish, 
                                                   Include_Bath, Version, T_Mains, Day_Of_Year, Temperature_Bath, Temperature_Shower, SquareFootage_Dwelling, 
                                                   Water, Distribution_System_Type)

start_date = datetime(2022, 1, 1, 0, 0, 0) # The date at the start of the draw profile
end_date = datetime(2022, 12, 31, 0, 0, 0)

current_date = start_date
us_holidays = holidays.US()

all_data = pd.DataFrame()

occupants = 1
variant = 0
while current_date < end_date:
    Day_Of_Year = current_date.timetuple().tm_yday
    # print(Day_Of_Year)
    
    if current_date in us_holidays:
        # print("holiday")
        profile_name = f"{occupants}H{variant}"
        _profile = create_one_profile(profile_name, Day_Of_Year)
    # Check if the date is a weekend
    elif current_date.weekday() == 5 or current_date.weekday() == 6:
        # print("It's a weekend!")
        profile_name = f"{occupants}E{variant}"
        _profile = create_one_profile(profile_name, Day_Of_Year)
    else:
        # print("It's not a weekend.")
        profile_name = f"{occupants}D{variant}"
        _profile = create_one_profile(profile_name, Day_Of_Year)

    current_date += timedelta(days=1) # Increment the date by one day
    converted_profile = Convert_Profile_SingleDay(_profile[profile_name], Day_Of_Year, Timestep, ClimateZone)

    all_data = pd.concat([all_data, converted_profile])
    # if current_date in us_holidays:
    #         print(f"{current_date} is a US holiday: {us_holidays.get(current_date)}")
all_data.to_csv(os.path.join(os.getcwd(), 'DrawProfiles', '{}.csv'.format(f"{occupants}Y{variant}")))

Day_Of_Year = current_date.timetuple().tm_yday
# print('Creating profiles')
# Profiles = Create_Hot_Profiles('Single', Profile_List[0], ClimateZone, Include_Faucet, Include_Shower, Include_Clothes, Include_Dish, 
#                                 Include_Bath, Version, T_Mains, Day_Of_Year, Temperature_Bath, Temperature_Shower, SquareFootage_Dwelling, 
#                                 Water, Distribution_System_Type)

# print('Converting profiles to timestep-based')
# for key in Profiles.keys():
#     print("key", key)
#     print(">>>>", Profiles)
#     print("Profile", Profiles[key])
#     Profiles[key] = Convert_Profile_SingleDay(Profiles[key], Day_Of_Year, Timestep, ClimateZone)

# print('Saving draw profiles to .csv')
# for key in Profiles.keys():
#     Profiles[key].to_csv(os.path.join(os.getcwd(), 'DrawProfiles', '{}.csv'.format(key)))
    

In [ ]:
from datetime import datetime, timedelta
import holidays

start_date = datetime(2022, 1, 1, 0, 0, 0)
end_date = datetime(2023, 1, 1, 0, 0, 0)

current_date = start_date
us_holidays = holidays.US()

while current_date < end_date:
    Day_Of_Year = current_date.timetuple().tm_yday

    # Check if the date is a weekend
    # if current_date.weekday() == 5 or current_date.weekday() == 6:
    #     print("It's a weekend!")
    # else:
    #     print("It's not a weekend.")
    if current_date in us_holidays:
            print(f"{current_date} is a US holiday: {us_holidays.get(current_date)}")
    current_date += timedelta(days=1)

    